# Práce s datovými typy a objekty pro časovou analýzu v Pandas

Pandas poskytuje robustní nástroje pro práci s časovými daty, které umožňují snadnou manipulaci a analýzu časových sérií. Níže jsou uvedeny základní třídy a typy objektů pro práci s časovými údaji, včetně metod pro jejich vytvoření.

| Koncept           | Skalární Třída | Třída objektu / indexu | Datový Typ v pandas      | Primární metoda vytvoření    |
|-------------------|----------------|------------------------|--------------------------|------------------------------|
| **Datum a čas**   | `Timestamp`    | `DatetimeIndex`        | `datetime64[ns]` nebo `datetime64[ns, tz]` | `to_datetime` nebo `date_range` |
| **Časová delta**  | `Timedelta`    | `TimedeltaIndex`       | `timedelta64[ns]`        | `to_timedelta` nebo `timedelta_range` |
| **Časové rozpětí**| `Period`       | `PeriodIndex`          | `period[freq]`           | `Period` nebo `period_range` |
| **Datumové posuny** | `DateOffset` | None                   | None                     | `DateOffset`                 |

> Zdroj: [Pandas Time Series Reference](https://pandas.pydata.org/docs/user_guide/timeseries.html)

## Klíčové metody pro práci s časovými daty



### Metoda `to_datetime()`

- Tato metoda převádí vstupní hodnotu na typ `datetime64[ns]`, případně `datetime64[ns, tz]`, pokud je hodnota specifikována v časové zóně.
- **Rok** musí být uveden jako čtyřmístný číselný řetězec.
- **Výchozí formát**: Měsíc předchází roku, ale toto pořadí lze upravit pomocí parametru `dayfirst`.
- **Vlastní formát**: Pokud datum není ve standardním formátu, lze použít parametr `format` k určení formátu data. Více o formátování najdete [zde](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).
- **Časová zóna**: Časová informace se výchozím nastavením vrací podle systémového času počítače. Pro návrat v UTC lze využít parametr `utc`.
- **Změna časové zóny**: Pro převod časové zóny lze použít metody `tz_convert()` nebo `tz_localize()`. Podrobnosti o metodě jsou dostupné [zde](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp). Přehled časových zón poskytuje knihovna `pytz`, viz [pytz na PyPI](https://pypi.org/project/pytz/).


In [7]:
import pandas as pd  # Import knihovny pandas pro práci s daty

# Definice data z řetězce ve formátu 'yy-mm-dd'
datum = pd.to_datetime('22-01-15', format='%y-%m-%d')

# Definice data a času z řetězce ve standardním formátu 'yyyy-mm-dd hh:mm'
datum_cas = pd.to_datetime('2022-01-15 15:45')

# Definice data a času z aktuálního data s použitím UTC časové zóny
datum_cas_dnes = pd.to_datetime('today', utc=True)

# Definice data a času z aktuálního data s časem definovaným uživatelem,
# pokud je jako argument použit pouze čas, použije se dnešní datum
datum_cas_dnes2 = pd.to_datetime('15:45')

# Výpis definovaných dat a časů
print('datum cas:', datum_cas)
print('datum cas dnes:', datum_cas_dnes)
print('datum cas dnes2:', datum_cas_dnes2)


datum cas: 2022-01-15 15:45:00
datum cas dnes: 2024-03-24 16:25:14.606540+00:00
datum cas dnes2: 2024-03-24 15:45:00


In [8]:
import pandas as pd  # Import knihovny pandas pro práci s daty

# Vytvoření 'aware' Timestamp pro aktuální datum a čas v UTC
datum_cas_dnes_utc = pd.to_datetime('today', utc=True)

# Konverze tohoto Timestamp na časové pásmo Moskvy
datum_cas_dnes_moscow = datum_cas_dnes_utc.tz_convert('Europe/Moscow')

# Výpis obou Timestampů s příslušným časovým pásmem
print(f"UTC Timestamp: {datum_cas_dnes_utc}")  # Výpis času v UTC
print(f"Moscow Timestamp: {datum_cas_dnes_moscow}")  # Výpis převedeného času na časové pásmo Moskvy


UTC Timestamp: 2024-03-24 16:25:14.622836+00:00
Moscow Timestamp: 2024-03-24 19:25:14.622836+03:00


### Metoda `to_timedelta()`

- Tato metoda převádí vstupní hodnotu na typ `timedelta64[ns]`.
- **Význam**: Reprezentuje časový rozdíl mezi dvěma údaji nebo posun v čase.
- **Aritmetické operace**: Lze jednoduše přičítat nebo odečítat od objektů typu `datetime`, čímž se posouvá čas.
- **Možné jednotky posunu**:
  - `ns` - nanosekundy
  - `us` - mikrosekundy
  - `ms` - milisekundy
  - `s` - sekundy
  - `m` - minuty
  - `h` - hodiny
  - `D` - dny
  - `W` - týdny

Podrobný přehled všech možných jednotek a aliasů najdete [zde](https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html#pandas.Timedelta).

> **Poznámka**: Objekty typu `timedelta` se vytvoří automaticky při odečítání dvou objektů `datetime64`.


In [9]:
import pandas as pd  # Import knihovny pandas pro práci s daty

# Vytvoření 'naive' Timestamp pro 'dnes' s explicitním nastavením na UTC
datum_cas_dnes = pd.to_datetime('today', utc=True)

# Vytvoření Timedelta pro posun o 3 dny, 2 hodiny a 3 sekundy
posun = pd.to_timedelta('3 days 2 hours 3 seconds')

# Posunutí času o zadanou Timedelta
posunuto = datum_cas_dnes + posun

# Výpis aktuálního data a času a data a času po posunu
print('Datum a cas dnes:', datum_cas_dnes)
print('Posunuto:', posunuto)


Datum a cas dnes: 2024-03-24 16:25:14.625285+00:00
Posunuto: 2024-03-27 18:25:17.625285+00:00


### Metoda `period()`

- Tato metoda je určena pro práci s objekty typu `datetime64[ns]`.
- **Převod na `period[freq]`**: Umožňuje převést vstupní hodnotu na typ `period[freq]`, který nepředstavuje konkrétní časový bod, ale časový interval. Například při nastavení frekvence `Y` představuje celý rok, a můžeme se odkazovat na jeho jednotlivé části.
- **Frekvence (freq)**: Může být nastavena pomocí různých aliasů, které specifikují časové rozlišení (např. rok, měsíc, den). Kompletní seznam těchto aliasů najdete [zde](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-period-aliases).
- **Změna frekvence**: Metoda `asfreq()` umožňuje převést objekt `Period` na jinou frekvenci. Více informací o této metodě je dostupných [zde](https://pandas.pydata.org/docs/reference/api/pandas.Period.html#pandas.Period).

> Další informace o `Period` objektu naleznete v [oficiální dokumentaci](https://pandas.pydata.org/docs/reference/api/pandas.Period.html).


In [10]:
import pandas as pd  # Import knihovny pandas pro práci s daty
import datetime  # Import modulu datetime pro práci s datem a časem

# Výpočet dnešního datumu a času pomocí datetime
dnes = datetime.datetime.now()
# Konverze dnešního datumu a času na pandas Period s denní frekvencí
dnes_perioda = pd.Period(dnes, freq='D')

# Výpis typů a hodnot pro dnešní datum a čas a pro dnešní periodu
print('Dnes:', type(dnes), dnes)
print('Dnes perioda:', type(dnes_perioda), dnes_perioda)

# Získání čísla dne v roce z dnešní periody
den_roku = dnes_perioda.dayofyear

# Výpis typu pro proměnnou dnes a čísla dne v roce
print('Den roku:', type(dnes), den_roku)


Dnes: <class 'datetime.datetime'> 2024-03-24 16:25:14.636250
Dnes perioda: <class 'pandas._libs.tslibs.period.Period'> 2024-03-24
Den roku: <class 'datetime.datetime'> 84


### Metoda `DateOffset()`

- Tato metoda slouží k definování posunu v čase.
- **Posun v čase**: `DateOffset` umožňuje provádět časový posun v kalendářních jednotkách, jako je den nebo měsíc. Například ji lze použít pro kalendářní posuny, jako je přesun na příští měsíc nebo rok.
- **Požadavek na časový index**: Při použití `DateOffset` s metodou `shift` v DataFrame musí být časová osa nastavena jako index. Například posun o rok s `df['sloupec'].shift(1, freq=pd.DateOffset(years=1))` vyžaduje, aby byl sloupec s datem nastaven jako index.
- **Příklad využití**: `pd.to_datetime('today') + pd.DateOffset(days=1)` vrací zítřejší datum.
- **Porovnání s `timedelta`**: `timedelta` poskytuje přesný časový posun (např. přesně 24 hodin), zatímco `DateOffset` se používá tam, kde je požadován posun v kalendářních jednotkách (např. posun na příští měsíc nebo rok).
- **Možné jednotky posunu**:
  - `years` - roky
  - `months` - měsíce
  - `weeks` - týdny
  - `days` - dny
  - `hours` - hodiny
  - `minutes` - minuty
  - `seconds` - sekundy
  - `microseconds` - mikrosekundy
  - `nanoseconds` - nanosekundy

Kompletní přehled dostupných aliasů a jednotek je k dispozici v [dokumentaci](https://pandas.pydata.org/docs/reference/api/pandas.DateOffset.html#pandas.DateOffset).


In [11]:
import pandas as pd  # Import knihovny pandas pro práci s daty
import datetime  # Import modulu datetime pro práci s datem a časem

# Získání aktuálního data a času
dnes = datetime.datetime.now()
# Výpočet data začátku týdne (pondělí) pomocí pandas offsetu
zacatek_tydne = dnes - pd.offsets.Week(weekday=0)

# Výpis data začátku týdne
print('Zacatek tydne:', zacatek_tydne)


Zacatek tydne: 2024-03-18 16:25:14.643495


In [12]:
import pandas as pd  # Import knihovny pandas pro práci s datovými rámci

# Načtení datového souboru CSV z URL s určením delimiteru ';'
df = pd.read_csv('https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv', delimiter=';')

# Výpis datových typů sloupců v DataFrame
print(f'Datové typy sloupců: {df.dtypes}')

# Ponechání pouze sloupce DATE a konverze na DataFrame
df = df['DATE'].to_frame()
# Převod řetězců na datový typ datetime
df['DATUM'] = pd.to_datetime(df['DATE'])
# Vytvoření timedelta pro posun o jeden den
df['POSUN DEN'] = pd.to_timedelta('1D')
# Výpočet data o jeden den dříve
df['DATUM-1'] = df['DATUM'] - df['POSUN DEN']
# Vytvoření offsetu pro posun na začátek týdne
df['POSUN NA ZACATEK TYDNE'] = pd.offsets.Week(weekday=0)
# Vytvoření offsetu pro posun o jeden rok dozadu
df['POSUN O 1 ROK'] = pd.offsets.DateOffset(years=-1)
# Výpočet data odpovídajícího začátku týdne
df['ZACATEK TYDNE'] = df['DATUM'].apply(lambda x: x - pd.offsets.Week(weekday=0))
# Výpočet data o jeden rok zpět
df['DATUM-1 ROK'] = df['DATUM'].apply(lambda d: d - pd.offsets.DateOffset(years=1))

# Zobrazení prvních pěti řádků upraveného DataFrame
df.head()


Datové typy sloupců: STATION     object
NAME        object
DATE        object
PRCP       float64
SNWD       float64
TAVG       float64
TMAX       float64
TMIN       float64
dtype: object


DATE      DATUM POSUN DEN    DATUM-1 POSUN NA ZACATEK TYDNE  \
0  2006-12-01 2006-12-01    1 days 2006-11-30      <Week: weekday=0>   
1  2006-12-02 2006-12-02    1 days 2006-12-01      <Week: weekday=0>   
2  2006-12-03 2006-12-03    1 days 2006-12-02      <Week: weekday=0>   
3  2006-12-04 2006-12-04    1 days 2006-12-03      <Week: weekday=0>   
4  2006-12-05 2006-12-05    1 days 2006-12-04      <Week: weekday=0>   

            POSUN O 1 ROK ZACATEK TYDNE DATUM-1 ROK  
0  <DateOffset: years=-1>    2006-11-27  2005-12-01  
1  <DateOffset: years=-1>    2006-11-27  2005-12-02  
2  <DateOffset: years=-1>    2006-11-27  2005-12-03  
3  <DateOffset: years=-1>    2006-11-27  2005-12-04  
4  <DateOffset: years=-1>    2006-12-04  2005-12-05

### Cvičení 1

1. Do proměnné `dnes` si spočítejte dnešní datum
2. Do proměnné `dnes_perioda` si uložte hodnotu proměnné `dnes` převedenou na Periodu s frekvencí Den
3. Do proměnné `dnes_tyden` si spočítejte číslo dneška v roce (tj. kolikátý den roce je dnešek)

### Cvičení 2

1. Do proměnné df nahrajte data z github se srážkami.
2. Převeďte sloupec [DATE] na typ datetime64[ns].
3. Dopočítejte sloupec [ZACATEK TYDNE], ve kterém bude dopočteno datum začátku týdne hodnoty ze sloupce [DATE].
4. Dopočítejte sloupec [PREDCHOZI TYDEN], ve kterém bude dopočteno datum začátku týdne hodnoty ze sloupce [DATE] o týden zpět.


### Cvičení3
1. Do proměnné df nahrajte data z github se srážkami.
2. Převeďte sloupec [DATE] na typ datetime64[ns].
3. Do proměnné df_datumy vytvořte date_range od minimální hodnoty sloupce [DATE] do maximální hodnoty sloupce [DATE].
4. Do proměnné df_agregovano agregujte df dle sloupce [DATE], kdy nás zajímá počet (count) hodnot ve sloupci [NAME].
5. Spojte zleva dataframe df_datumy a df_agregovano dle sloupce.
6. Určete, pro které [DATE] nemáme žádné hodnoty (tj. žádná stanice nevykázala informace o povětrnostních podmínkách).



### Cvičení 4

1. Do proměnné df nahrajte data z github se srážkami.
2. Převeďte sloupec [DATE] na typ datetime64[ns]
3. Dopočtete sloupec [ROK]
4. Převeďte hodnotu ze sloupce [ROK] na Periodu rok
5. Do sloupce [ROK START] dopočítejte začátek roku